In [54]:
cd nli/infersent/

[Errno 2] No such file or directory: 'nli/infersent/'
/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [55]:
import torch
import numpy as np

GLOVE_PATH = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'

In [11]:
model_path = 'saved_models/infersent.allnli.pickle'
model = torch.load(model_path, map_location=lambda storage, loc: storage)

In [12]:
model.set_glove_path(GLOVE_PATH)

In [13]:
sentences = [
    'This is a sample sentence.',
    'The cat likes to walk.',
    'The cat likes to talk.',
    'The dog likes to talk.',
    'This is a completely unrelated sentence.'
]

In [15]:
model.build_vocab(sentences)

Found 17(/17) words with glove vectors
Vocab size : 17


In [18]:
embeddings = model.encode(sentences, bsize=2, tokenize=False, verbose=True)

Nb words kept : 31/36 (86.11 %)
Speed : 7.6 sentences/s (cpu mode, bsize=2)


In [20]:
embeddings.shape

(5, 4096)

In [45]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def sim(sent1, sent2):
    em = model.encode([sent1, sent2], tokenize=False, verbose=True)
    return cosine(em[0], em[1])

In [53]:
model.get_word_dict(sentences).keys()

['a',
 'sentence',
 'This',
 'is',
 'walk',
 'dog',
 '.',
 'sample',
 'to',
 'completely',
 'unrelated',
 'likes',
 'The',
 'cat',
 '</s>',
 'talk',
 '<s>']

## NLI

In [82]:
run data.py

In [140]:
import torch
import numpy as np
from torch.autograd import Variable
from nltk.tokenize import word_tokenize

GLOVE_PATH = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'

In [217]:
s1 = [
    'I do not want to go home.',
    'He likes butter.',
    'He never speaks loud.',
    'The weather is good',
    'What is the matter?',
    'He never speaks loud.',
    'He made two mistakes',
]
s2 = [
    'I wish to go home.',
    'He hates butter.',
    'He never speaks soft.',
    'The weather is bad',
    'What is the matter with you?',
    'He always speaks soft.',
    'He made mistakes twice.',
]

In [218]:
word_dict = get_word_dict(s1 + s2)
word_vec = get_glove(word_dict, GLOVE_PATH)

Found 36(/36) words with glove vectors


In [219]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()
outputs = []
for i in range(len(s1)):
    ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
    ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
    # ss1 = sorted(ss1, key = lambda x: len(x), reverse=True)
    # ss2 = sorted(ss2, key = lambda x: len(x), reverse=True)
    s1_batch, s1_len = get_batch(ss1, word_vec)
    s2_batch, s2_len = get_batch(ss2, word_vec)
    s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
    output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
    outputs.append(output.data.max(1)[1].numpy()[0])
outputs

[2, 2, 0, 2, 0, 2, 0]

In [220]:
s1, s2

(['I do not want to go home.',
  'He likes butter.',
  'He never speaks loud.',
  'The weather is good',
  'What is the matter?',
  'He never speaks loud.',
  'He made two mistakes'],
 ['I wish to go home.',
  'He hates butter.',
  'He never speaks soft.',
  'The weather is bad',
  'What is the matter with you?',
  'He always speaks soft.',
  'He made mistakes twice.'])

## Prostrcustus transformation matrix

In [2]:
cd ~/acads/11797/bioasq/nli/infersent/data/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent/data


In [3]:
import numpy as np

In [4]:
mat = np.load('matmul_glove_med.npy')

In [22]:
mat.shape

(300, 200)

In [23]:
U, s, V = np.linalg.svd(mat, full_matrices=False)

In [24]:
U.shape, s.shape, V.shape

((300, 200), (200,), (200, 200))

In [33]:
tm = U.dot(V)

In [37]:
np.save('prostrcustus_transformation_matrix.npy', tm)

## Embedding Creation

In [1]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [2]:
from dataLoader import DataLoader

input_path = 'input/BioASQ-trainingDataset6b.json'
# input_path = 'input/BioASQ-task6bPhaseB-testset3.json'
data = DataLoader(input_path)

In [3]:
pubmed_vectors_file = '/Volumes/Nitish-Passport/bioasq_files/wikipedia-pubmed-and-PMC-w2v.bin'

input_glove_file = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/%s_glove.840B.300d.txt' % data.name

In [4]:
cd ~/acads/11797/bioasq/nli/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli


In [5]:
run hypotheses.py

In [6]:
# RerankingParser._parser_model_loaded = False
# rrp = RerankingParser.from_unified_model_dir('/Users/nitish/.local/share/bllipparser/GENIA+PubMed')
# # fetch_and_load('GENIA+PubMed', verbose=True)
# yesno = data.get_questions_of_type('yesno')
# for q in tqdm(yesno):
#     q.assertion_pos = q2s(q.question, rrp)
set_assertions_for_yesno_questions(data)

Model directory: /Users/nitish/.local/share/bllipparser/GENIA+PubMed
Model directory already exists, not reinstalling


100%|██████████| 616/616 [01:21<00:00,  7.52it/s]


In [7]:
yesno = data.get_questions_of_type('yesno')
all_sentences = []
for q in yesno:
    all_sentences.append(q.assertion_pos)
    all_sentences += q.snippet_sentences

In [8]:
cd ~/acads/11797/bioasq/nli/infersent

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [9]:
run data.py

In [10]:
word_dict = get_word_dict(all_sentences)
tokens = word_dict.keys()

In [120]:
run create_glove_embeddings.py

In [65]:
# pubtator_vectors = word2vec.load(pubmed_vectors_file)

In [77]:
# import cPickle as pickle

# with open('/Users/nitish/acads/11797/bioasq/nli/infersent/data/uncommon_vocab.pkl', 'rb') as fp:
#     uncommon_vocab = pickle.load(fp)

In [123]:
k = 0
with open(new_glove_file, 'w') as fp:
    for word in tokens:
        k += 1
        if k % 5000 == 0:
            print('Finished %d words' % k)
        if word in uncommon_vocab and word in pubtator_vectors:
            vec = tm.dot(pubtator_vectors[word][:, np.newaxis])[:, 0]
            line = ' '.join([word] + [str(i) for i in vec])
            fp.write('%s\n' % line)

In [87]:
vec = tm.dot(pubtator_vectors['localizes'][:, np.newaxis])[:, 0].shape
vec.lot_list()

(300,)

## Run infersent on yesno questions

In [1]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [2]:
run dataLoader.py

In [3]:
input_path = 'input/BioASQ-trainingDataset6b.json'
# input_path = 'input/BioASQ-task6bPhaseB-testset3.json'
data = DataLoader(input_path)

input_glove_file = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/%s_glove.840B.300d.txt' % data.name
yesno_ans_file = 'output/yesno_%s.json' % data.name

In [4]:
cd ~/acads/11797/bioasq/nli/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli


In [5]:
run hypotheses.py

In [6]:
set_assertions_for_yesno_questions(data)

yesno = data.get_questions_of_type('yesno')
all_sentences = []
for q in yesno:
    all_sentences.append(q.assertion_pos)
    all_sentences += q.snippet_sentences

Model directory: /Users/nitish/.local/share/bllipparser/GENIA+PubMed
Model directory already exists, not reinstalling


100%|██████████| 616/616 [01:36<00:00,  6.40it/s]


In [7]:
cd ~/acads/11797/bioasq/nli/infersent

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [8]:
run data.py

In [9]:
word_dict = get_word_dict(all_sentences)
word_vec = get_glove(word_dict, input_glove_file, new_glove_file)

Found 22093(/23268) words with glove vectors


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [22]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()

for q in yesno[:1]:
    outputs = []
    sentences = [i['text'] for i in q.ranked_sentences()[:20]]
    s1, s2 = sentences, [q.assertion_pos] * len(sentences)
#     s2, s1 = sentences, [q.assertion_pos] * len(sentences)
#     if q.exact_answer_ref == 'yes':
#         continue
    total = 0
    for i in range(len(s1)):
        ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
        ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
        # ss1 = sorted(ss1, key = lambda x: len(x), reverse=True)
        # ss2 = sorted(ss2, key = lambda x: len(x), reverse=True)
        s1_batch, s1_len = get_batch(ss1, word_vec)
        s2_batch, s2_len = get_batch(ss2, word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
        total += output
        outputs.append(output.data.max(1)[1].numpy()[0])
    print (np.array(outputs) == 0).sum(), (np.array(outputs) == 2).sum(), q.exact_answer_ref, total.data.max(1)[1].numpy()[0]

8 0 yes 0


Variable containing:
 1.5178 -0.6057 -0.9143
[torch.FloatTensor of size 1x3]

## Eval

In [20]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()

def get_answer(n_entail, n_contradict):
    if n_entail > n_contradict:
        return 'yes'
    elif n_contradict > n_entail:
        return 'no'
    else:
        if n_entail == 0:
            return 'no'
        else:
            return 'yes'

entailments = []
for q in tqdm(yesno):
    outputs = []
    sentences = [i['text'] for i in q.ranked_sentences()]
    s1, s2 = sentences, [q.assertion_pos] * len(sentences)
    total = 0
    for i in range(len(s1)):
        ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
        ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
        s1_batch, s1_len = get_batch(ss1, word_vec)
        s2_batch, s2_len = get_batch(ss2, word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
        total += output
        outputs.append(output.data.max(1)[1].numpy()[0])
    n_entail = (np.array(outputs) == 0).sum()
    n_contradict = (np.array(outputs) == 2).sum()
    total_res = total.data.max(1)[1].numpy()[0] if isinstance(total, Variable) else 0
    q.exact_answer = get_answer(n_entail, n_contradict)
    entailments.append((n_entail, n_contradict, total_res, q.exact_answer))

data.eval_yesno()

100%|██████████| 616/616 [4:09:53<00:00, 24.34s/it]  

Total Accuracy = 0.590909 (364 / 616)
YES Accuracy = 0.583969 (306 / 524)
No Accuracy = 0.630435 (58 / 92)


In [21]:
sentences

['Sonidegib phosphate: new approval for basal cell carcinoma',
 'Sonidegib, a novel smoothened inhibitor for the treatment of advanced basal cell carcinoma',
 'Sonidegib (Odomzo), an oral smoothened (SMO) antagonist, is indicated for the treatment of adults with locally advanced basal cell carcinoma (laBCC) who are not candidates for surgery or radiation therapy, or adults with recurrent laBCC following surgery or radiation therapy',
 'Oral sonidegib is approved in Switzerland for the treatment of adult patients with advanced basal cell carcinoma (BCC) and in the US and EU for the treatment of adult patients with locally advanced BCC that has recurred following surgery or radiation therapy, or those who are not candidates for surgery or radiation therapy',
 'Oral Smoothened (Smo) inhibitors Vismodegib, Sonidegib, and Taladegib have shown to be effective in several trials',
 'The hedgehog pathway inhibitor sonidegib demonstrated meaningful tumor shrinkage in more than 90% of patients wi

In [20]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [21]:
# data.save_yesno_answers(yesno_ans_file)
# data.load_answers_from_file(yesno_ans_file, True)

In [31]:
q = yesno[5]
print q.question
print q.exact_answer
for i in q.ranked_sentences():
    print i['text']

Is there an association between Klinefelter syndrome and breast cancer?
yes
Male breast cancer risk factors show strong association with BRCA2 mutations, as well as Klinefelter syndrome
Screening for breast cancer in male-to-female transsexuals should be undertaken for those with additional risk factors (e.g., family history, BRCA2 mutation, Klinefelter syndrome) and should be available to those who desire screening, preferably in a clinical trial
Patients with 47, XXY karyotype (Klinefelter syndrome) appear to have increased risk of developing cancer, especially male breast cancer, germ cell tumours and non Hodgkin lymphomas, but rarely acute myeloid leukaemia
Major risk factors for developing male BC include clinical disorders involving hormonal imbalances (excess of estrogen or a deficiency of testosterone as seen in patients with Klinefelter syndrome) and a positive family history for breast cancer
The main risk factors include: the mutation of genes BRCA 1 and 2, Klinefelter's syn